# Preliminary EDA and Data Prep for Census Data File

This project is divided into 3 separate notebooks, one for each section of our analysis:

<li>*Part 1:* FBI NICS Data and patterns within the guns data - observable changes over time, cyclical nature of permits.
<li>*Part 2:* Census data wrangling and cleaning and creation of a single dataset that can be used for Part 3.
<li>*Part 3:* A look at the relationship between variables in the census data and the normalized number of background checks - i.e. are there characteristics of a state’s demographics that appear to correlate to the number of background checks? This section will be performed for a subset of data from the NICS checks so we can compare it to the census data.

##### *This notebook contains Part 2*

In [410]:
# Imports and files
# Pagckages Import Section
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

# Looking at easier ways of converting strings to dates.
import timestring as ts
#da = ts.Date("2017-09")
% matplotlib inline

df = pd.read_csv('US_Census.csv')
#df.head()

### Useful Functions Section

In [411]:
def get_real_types(df):
    '''Input a pandas dataframe and get back a dictionary showing the 
    actual data types of each column - instead of <object> pointers.
    ** make sure you include from pprint import pprint
    
    '''

    # Create empty dictionary:
    types_list = {}
    # Create a list of the columns in df
    columns_list = df.columns.tolist()
    
    # For loop creates a key-value pair in the types_list dict with column name and the
    # data type of the value in the first row for that column

    for i in columns_list:
        types_list[i] = type(df[i].iloc[0])

    pprint(types_list)

    return(types_list)

In [412]:
# Basic info about the data set:
#df.info()
#get_real_types(df)

From this basic exploration, we can see that this data is not going to be that useful - all the values are strings. Also, the columns are states instead of the attributes of each state (like the layout of the NICS data). 

To-Do's for this data:

- drop the rows that are not really part of the data (65-84). We'll keep these in a backup copy of the file in case we need to refer to them.
- drop the Fact note column
- transform the data so the rows and columns are transposed.
#df = df.T
- rename the columns
- change all the data to floats so that we will be able to use it

In [413]:
# Drop the data we definitely don't need or that doesn't belong
df.drop(df.index[65:85], inplace=True)
df.drop(columns = 'Fact Note', inplace=True)
df.reset_index(drop=False)
df

,Fact,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,"Population estimates, July 1, 2016, (V2016)","4,863,300","741,894","6,931,071","2,988,248","39,250,017","5,540,545","3,576,452","952,065","20,612,439",...,865454,6651194,"27,862,596","3,051,217","624,594","8,411,808","7,288,000","1,831,102","5,778,708","585,501"
1,"Population estimates base, April 1, 2010, (V2...","4,780,131","710,249","6,392,301","2,916,025","37,254,522","5,029,324","3,574,114","897,936","18,804,592",...,814195,6346298,"25,146,100","2,763,888","625,741","8,001,041","6,724,545","1,853,011","5,687,289","563,767"
2,"Population, percent change - April 1, 2010 (es...",1.70%,4.50%,8.40%,2.50%,5.40%,10.20%,0.10%,6.00%,9.60%,...,0.063,0.048,10.80%,10.40%,-0.20%,5.10%,8.40%,-1.20%,1.60%,3.90%
3,"Population, Census, April 1, 2010","4,779,736","710,231","6,392,017","2,915,918","37,253,956","5,029,196","3,574,097","897,934","18,801,310",...,814180,6346105,"25,145,561","2,763,885","625,741","8,001,024","6,724,540","1,852,994","5,686,986","563,626"
4,"Persons under 5 years, percent, July 1, 2016, ...",6.00%,7.30%,6.30%,6.40%,6.30%,6.10%,5.20%,5.80%,5.50%,...,0.071,0.061,7.20%,8.30%,4.90%,6.10%,6.20%,5.50%,5.80%,6.50%
5,"Persons under 5 years, percent, April 1, 2010",6.40%,7.60%,7.10%,6.80%,6.80%,6.80%,5.70%,6.20%,5.70%,...,0.073,0.064,7.70%,9.50%,5.10%,6.40%,6.50%,5.60%,6.30%,7.10%
6,"Persons under 18 years, percent, July 1, 2016,...",22.60%,25.20%,23.50%,23.60%,23.20%,22.80%,21.10%,21.50%,20.10%,...,0.246,0.226,26.20%,30.20%,19.00%,22.20%,22.40%,20.50%,22.30%,23.70%
7,"Persons under 18 years, percent, April 1, 2010",23.70%,26.40%,25.50%,24.40%,25.00%,24.40%,22.90%,22.90%,21.30%,...,0.249,0.236,27.30%,31.50%,20.70%,23.20%,23.50%,20.90%,23.60%,24.00%
8,"Persons 65 years and over, percent, July 1, 2...",16.10%,10.40%,16.90%,16.30%,13.60%,13.40%,16.10%,17.50%,19.90%,...,0.16,0.157,12.00%,10.50%,18.10%,14.60%,14.80%,18.80%,16.10%,15.00%
9,"Persons 65 years and over, percent, April 1, 2010",13.80%,7.70%,13.80%,14.40%,11.40%,10.90%,14.20%,14.40%,17.30%,...,0.143,0.134,10.30%,9.00%,14.60%,12.20%,12.30%,16.00%,13.70%,12.40%


In [414]:
# Transpose the data so that the rows are observations for each state
#df_census = df.T
# Set the column names to the values in the Fact row

df_census.columns = df_census.iloc[0]
df_census.head()
df_census.to_csv('census_by_state.csv', index=True)

# Let's get some more info on what data is in each column in order to see which variables
# we might consider looking at for this analysis.

#df_census.head()
df_census.columns

Index(['Population estimates, July 1, 2016,  (V2016)',
       'Population estimates base, April 1, 2010,  (V2016)',
       'Population, percent change - April 1, 2010 (estimates base) to July 1, 2016,  (V2016)',
       'Population, Census, April 1, 2010',
       'Persons under 5 years, percent, July 1, 2016,  (V2016)',
       'Persons under 5 years, percent, April 1, 2010',
       'Persons under 18 years, percent, July 1, 2016,  (V2016)',
       'Persons under 18 years, percent, April 1, 2010',
       'Persons 65 years and over, percent,  July 1, 2016,  (V2016)',
       'Persons 65 years and over, percent, April 1, 2010',
       'Female persons, percent,  July 1, 2016,  (V2016)',
       'Female persons, percent, April 1, 2010',
       'White alone, percent, July 1, 2016,  (V2016)',
       'Black or African American alone, percent, July 1, 2016,  (V2016)',
       'American Indian and Alaska Native alone, percent, July 1, 2016,  (V2016)',
       'Asian alone, percent, July 1, 2016,  (V20

In [415]:
# Rename the columns:
df_census.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
# Create a list of the existing column names:
columns_list = df_census.columns.tolist()
pprint(columns_list)

['population_estimates,_july_1,_2016,__(v2016)',
 'population_estimates_base,_april_1,_2010,__(v2016)',
 'population,_percent_change_-_april_1,_2010_(estimates_base)_to_july_1,_2016,__(v2016)',
 'population,_census,_april_1,_2010',
 'persons_under_5_years,_percent,_july_1,_2016,__(v2016)',
 'persons_under_5_years,_percent,_april_1,_2010',
 'persons_under_18_years,_percent,_july_1,_2016,__(v2016)',
 'persons_under_18_years,_percent,_april_1,_2010',
 'persons_65_years_and_over,_percent,__july_1,_2016,__(v2016)',
 'persons_65_years_and_over,_percent,_april_1,_2010',
 'female_persons,_percent,__july_1,_2016,__(v2016)',
 'female_persons,_percent,_april_1,_2010',
 'white_alone,_percent,_july_1,_2016,__(v2016)',
 'black_or_african_american_alone,_percent,_july_1,_2016,__(v2016)',
 'american_indian_and_alaska_native_alone,_percent,_july_1,_2016,__(v2016)',
 'asian_alone,_percent,_july_1,_2016,__(v2016)',
 'native_hawaiian_and_other_pacific_islander_alone,_percent,_july_1,_2016,__(v2016)',
 'tw

In [427]:
# Construct a data set that contains only the data we will be analyzing with with the NICS data.
# Note that we need to keep population columns for calcs

df_census2 = df_census[['population_estimates,_july_1,_2016,__(v2016)', 'population_estimates_base,_april_1,_2010,__(v2016)', 'population,_percent_change_-_april_1,_2010_(estimates_base)_to_july_1,_2016,__(v2016)', 'foreign_born_persons,_percent,_2011-2015', 'owner-occupied_housing_unit_rate,_2011-2015', 'median_gross_rent,_2011-2015', 'median_household_income_(in_2015_dollars),_2011-2015', 'persons__without_health_insurance,_under_age_65_years,_percent', 'women-owned_firms,_2012', 'minority-owned_firms,_2012', 'all_firms,_2012']].copy()
# These column names are a nightmare - let's rename them!
columns_list2 = df_census2.columns.tolist()
pprint(columns_list2)

['population_estimates,_july_1,_2016,__(v2016)',
 'population_estimates_base,_april_1,_2010,__(v2016)',
 'population,_percent_change_-_april_1,_2010_(estimates_base)_to_july_1,_2016,__(v2016)',
 'foreign_born_persons,_percent,_2011-2015',
 'owner-occupied_housing_unit_rate,_2011-2015',
 'median_gross_rent,_2011-2015',
 'median_household_income_(in_2015_dollars),_2011-2015',
 'persons__without_health_insurance,_under_age_65_years,_percent',
 'women-owned_firms,_2012',
 'minority-owned_firms,_2012',
 'all_firms,_2012']


In [429]:
#Copied the list from above and created the dictionary needed for the new column names:
new_col_names = {'population_estimates,_july_1,_2016,__(v2016)':'pop_16',
 'population_estimates_base,_april_1,_2010,__(v2016)':'pop_10',
 'population,_percent_change_-_april_1,_2010_(estimates_base)_to_july_1,_2016,__(v2016)':'pop_chg',
 'foreign_born_persons,_percent,_2011-2015':'foreign_born_pct',
 'owner-occupied_housing_unit_rate,_2011-2015':'owner-occ_housing',
 'median_gross_rent,_2011-2015':'med_gross_rent',
 'median_household_income_(in_2015_dollars),_2011-2015':'med_hh_income',
 'persons__without_health_insurance,_under_age_65_years,_percent':'no_health_ins',
 'women-owned_firms,_2012':'women-owned_firms',
 'minority-owned_firms,_2012':'min-owned_firms',
'all_firms,_2012': 'all_firms'}

df_census2.rename(columns=new_col_names, inplace=True)
# Drop the Fact row if it's in there
df_census2.drop('Fact', axis=0, inplace=True)


df_census3 = df_census2

df_census3.head()

Fact,pop_16,pop_10,pop_chg,foreign_born_pct,owner-occ_housing,med_gross_rent,med_hh_income,no_health_ins,women-owned_firms,min-owned_firms,all_firms
Alabama,"4,863,300","4,780,131",1.70%,3.50%,68.70%,$717,"$43,623",10.70%,"137,630","92,219","374,153"
Alaska,"741,894","710,249",4.50%,7.40%,63.40%,"$1,146","$72,515",15.50%,"22,141","13,688","68,032"
Arizona,"6,931,071","6,392,301",8.40%,13.50%,62.80%,$913,"$50,255",11.90%,"182,425","135,313","499,926"
Arkansas,"2,988,248","2,916,025",2.50%,4.70%,66.10%,$677,"$41,371",9.30%,"75,962","35,982","231,959"
California,"39,250,017","37,254,522",5.40%,27.00%,54.30%,"$1,255","$61,818",8.30%,"1,320,085","1,619,857","3,548,449"


In [430]:
#Now let's take a look at these data types and make sure they're what we need for the analysis.

get_real_types(df_census3)

{'all_firms': <class 'str'>,
 'foreign_born_pct': <class 'str'>,
 'med_gross_rent': <class 'str'>,
 'med_hh_income': <class 'str'>,
 'min-owned_firms': <class 'str'>,
 'no_health_ins': <class 'str'>,
 'owner-occ_housing': <class 'str'>,
 'pop_10': <class 'str'>,
 'pop_16': <class 'str'>,
 'pop_chg': <class 'str'>,
 'women-owned_firms': <class 'str'>}


{'all_firms': str,
 'foreign_born_pct': str,
 'med_gross_rent': str,
 'med_hh_income': str,
 'min-owned_firms': str,
 'no_health_ins': str,
 'owner-occ_housing': str,
 'pop_10': str,
 'pop_16': str,
 'pop_chg': str,
 'women-owned_firms': str}

In [431]:
#All the data are strings! This is not useful - we need floats.
# In order to do that, we'll first need to remove the non-numeric characters from the data:
cols = df_census3.columns.tolist()
# pass them to df.replace(), specifying each char and it's replacement:
df_census3[cols] = df_census3[cols].replace({'[\$,)]': '', ',': '', '%':'', ' ':''}, regex=True)
#backup the data set:
df_census3_copy = df_census3

df_census3.head()


Fact,pop_16,pop_10,pop_chg,foreign_born_pct,owner-occ_housing,med_gross_rent,med_hh_income,no_health_ins,women-owned_firms,min-owned_firms,all_firms
Alabama,4863300,4780131,1.70,3.50,68.70,717,43623,10.70,137630,92219,374153
Alaska,741894,710249,4.50,7.40,63.40,1146,72515,15.50,22141,13688,68032
Arizona,6931071,6392301,8.40,13.50,62.80,913,50255,11.90,182425,135313,499926
Arkansas,2988248,2916025,2.50,4.70,66.10,677,41371,9.30,75962,35982,231959
California,39250017,37254522,5.40,27.00,54.30,1255,61818,8.30,1320085,1619857,3548449


In [432]:
# Now we can try to do the conversion of the strings to floats:
df_census3[cols] = df_census3[cols].astype('float64')
#check that it worked:
df_census3.head()


Fact,pop_16,pop_10,pop_chg,foreign_born_pct,owner-occ_housing,med_gross_rent,med_hh_income,no_health_ins,women-owned_firms,min-owned_firms,all_firms
Alabama,4863300.0,4780131.0,1.7,3.5,68.7,717.0,43623.0,10.7,137630.0,92219.0,374153.0
Alaska,741894.0,710249.0,4.5,7.4,63.4,1146.0,72515.0,15.5,22141.0,13688.0,68032.0
Arizona,6931071.0,6392301.0,8.4,13.5,62.8,913.0,50255.0,11.9,182425.0,135313.0,499926.0
Arkansas,2988248.0,2916025.0,2.5,4.7,66.1,677.0,41371.0,9.3,75962.0,35982.0,231959.0
California,39250017.0,37254522.0,5.4,27.0,54.3,1255.0,61818.0,8.3,1320085.0,1619857.0,3548449.0


In [433]:
#Pause here - we've done a lot of work on this data - let's create a csv file:
df_census3.to_csv('census_floats.csv', index =True)

#We will re-perform the import at this point - this cell is risky because it contains calculations
#which if repeated, will corrupt our data.

df_census4 = pd.read_csv('census_floats.csv')
df_census4.rename(columns={'Unnamed: 0': 'state'}, inplace=True)
df_census4.head()
#df_census4.info()

# Add in some columns we need for the analysis:
# Proportions of women and minority owned firms:

df_census4['women-owned_prop'] = df_census4['women-owned_firms']/df_census4['all_firms']
df_census4['min-owned_prop'] = df_census4['min-owned_firms']/df_census4['all_firms']

# For consistency, let's express all percentage columns as proportions:
df_census4['pop_chg'] = df_census4['pop_chg']/100
df_census4['foreign_born_pct'] = df_census4['foreign_born_pct']/100
df_census4['owner-occ_housing'] = df_census4['owner-occ_housing']/100
df_census4['no_health_ins'] = df_census4['no_health_ins']/100

df_census4.to_csv('census_complete.csv', index=True)
df_census4.head()


,state,pop_16,pop_10,pop_chg,foreign_born_pct,owner-occ_housing,med_gross_rent,med_hh_income,no_health_ins,women-owned_firms,min-owned_firms,all_firms,women-owned_prop,min-owned_prop
0,Alabama,4863300.0,4780131.0,0.017,0.035,0.687,717.0,43623.0,0.107,137630.0,92219.0,374153.0,0.367844,0.246474
1,Alaska,741894.0,710249.0,0.045,0.074,0.634,1146.0,72515.0,0.155,22141.0,13688.0,68032.0,0.325450,0.201199
2,Arizona,6931071.0,6392301.0,0.084,0.135,0.628,913.0,50255.0,0.119,182425.0,135313.0,499926.0,0.364904,0.270666
3,Arkansas,2988248.0,2916025.0,0.025,0.047,0.661,677.0,41371.0,0.093,75962.0,35982.0,231959.0,0.327480,0.155122
4,California,39250017.0,37254522.0,0.054,0.270,0.543,1255.0,61818.0,0.083,1320085.0,1619857.0,3548449.0,0.372017,0.456497


In [434]:
# Now we'll bring in the relevant data from the NICS dataset and merge it in:
nics_merge = pd.read_csv('final_nics.csv')
state_politics = pd.read_csv('state_politics.csv')
df_census_final = pd.read_csv('census_complete.csv')


In [435]:
nics_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 5 columns):
state              55 non-null object
handguns_total     55 non-null float64
long_guns_total    55 non-null float64
other_total        55 non-null float64
totals             55 non-null int64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.2+ KB


In [436]:
state_politics.head()

,state,politics
0,Alabama,Red
1,Alaska,Red
2,Arizona,Red
3,Arkansas,Red
4,California,Blue


In [437]:
df_census_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 15 columns):
Unnamed: 0           50 non-null int64
state                50 non-null object
pop_16               50 non-null float64
pop_10               50 non-null float64
pop_chg              50 non-null float64
foreign_born_pct     50 non-null float64
owner-occ_housing    50 non-null float64
med_gross_rent       50 non-null float64
med_hh_income        50 non-null float64
no_health_ins        50 non-null float64
women-owned_firms    50 non-null float64
min-owned_firms      50 non-null float64
all_firms            50 non-null float64
women-owned_prop     50 non-null float64
min-owned_prop       50 non-null float64
dtypes: float64(13), int64(1), object(1)
memory usage: 5.9+ KB


In [438]:
master1 = pd.merge(df_census_final, state_politics, how='outer', on=['state', 'state'])

In [439]:
master = pd.merge(master1, nics_merge, how='outer', on=['state', 'state'])
master = master[master.state != 'Kentucky']

master.head()



,Unnamed: 0,state,pop_16,pop_10,pop_chg,foreign_born_pct,owner-occ_housing,med_gross_rent,med_hh_income,no_health_ins,women-owned_firms,min-owned_firms,all_firms,women-owned_prop,min-owned_prop,politics,handguns_total,long_guns_total,other_total,totals
0,0.0,Alabama,4863300.0,4780131.0,0.017,0.035,0.687,717.0,43623.0,0.107,137630.0,92219.0,374153.0,0.367844,0.246474,Red,135207.0,126076.0,467577.0,3643862
1,1.0,Alaska,741894.0,710249.0,0.045,0.074,0.634,1146.0,72515.0,0.155,22141.0,13688.0,68032.0,0.325450,0.201199,Red,44639.0,43557.0,13438.0,566738
2,2.0,Arizona,6931071.0,6392301.0,0.084,0.135,0.628,913.0,50255.0,0.119,182425.0,135313.0,499926.0,0.364904,0.270666,Red,201930.0,130644.0,188299.0,2292122
3,3.0,Arkansas,2988248.0,2916025.0,0.025,0.047,0.661,677.0,41371.0,0.093,75962.0,35982.0,231959.0,0.327480,0.155122,Red,99117.0,113970.0,107521.0,1651493
4,4.0,California,39250017.0,37254522.0,0.054,0.270,0.543,1255.0,61818.0,0.083,1320085.0,1619857.0,3548449.0,0.372017,0.456497,Blue,636552.0,544564.0,1671978.0,10186989


In [440]:
#Now that we have all the data together, our last step is to create some calculated columns to
#show the NICS data per capita (i.e. normalized for population)

a = master['pop_16']

master['total_NICS_pc'] = master['totals']/a
master['handguns_pc'] = master['handguns_total']/a
master['long_guns_pc'] = master['long_guns_total']/a
master['other_pc'] = master['other_total']/a

master.head()

,Unnamed: 0,state,pop_16,pop_10,pop_chg,foreign_born_pct,owner-occ_housing,med_gross_rent,med_hh_income,no_health_ins,...,min-owned_prop,politics,handguns_total,long_guns_total,other_total,totals,total_NICS_pc,handguns_pc,long_guns_pc,other_pc
0,0.0,Alabama,4863300.0,4780131.0,0.017,0.035,0.687,717.0,43623.0,0.107,...,0.246474,Red,135207.0,126076.0,467577.0,3643862,0.749257,0.027801,0.025924,0.096144
1,1.0,Alaska,741894.0,710249.0,0.045,0.074,0.634,1146.0,72515.0,0.155,...,0.201199,Red,44639.0,43557.0,13438.0,566738,0.763907,0.060169,0.058711,0.018113
2,2.0,Arizona,6931071.0,6392301.0,0.084,0.135,0.628,913.0,50255.0,0.119,...,0.270666,Red,201930.0,130644.0,188299.0,2292122,0.330702,0.029134,0.018849,0.027167
3,3.0,Arkansas,2988248.0,2916025.0,0.025,0.047,0.661,677.0,41371.0,0.093,...,0.155122,Red,99117.0,113970.0,107521.0,1651493,0.552663,0.033169,0.038139,0.035981
4,4.0,California,39250017.0,37254522.0,0.054,0.270,0.543,1255.0,61818.0,0.083,...,0.456497,Blue,636552.0,544564.0,1671978.0,10186989,0.259541,0.016218,0.013874,0.042598


In [442]:
#WOW IT'S DONE!! :) We now have a single dataframe we can use to do our analysis of the relationship
#between various census variables and the NICS data!

master.to_csv('master_NICS_2011-17.csv', index=False)
